# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235378516119                   -0.50    8.0
  2   -7.250412794343       -1.82       -1.41    1.0
  3   -7.251199180346       -3.10       -2.00    4.0
  4   -7.251002573451   +   -3.71       -1.90    4.0
  5   -7.251334995120       -3.48       -2.83    3.0
  6   -7.251338236553       -5.49       -3.19    3.0
  7   -7.251338787490       -6.26       -4.11    2.0
  8   -7.251338795550       -8.09       -4.30    4.0
  9   -7.251338798627       -8.51       -4.96    3.0
 10   -7.251338798672      -10.35       -5.30    1.0
 11   -7.251338798703      -10.50       -5.98    3.0
 12   -7.251338798704      -12.05       -6.29    2.0
 13   -7.251338798705      -12.68       -7.12    3.0
 14   -7.251338798705      -14.45       -7.25    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07294031242011863
[ Info: Arnoldi iteration step 2: normres = 0.5792058461468899
[ Info: Arnoldi iteration step 3: normres = 0.7831522124548206
[ Info: Arnoldi iteration step 4: normres = 0.27674708484663524
[ Info: Arnoldi iteration step 5: normres = 0.4529486047336903
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.06e-02, 6.43e-02, 3.69e-01, 2.53e-01, 1.78e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3153543972892036
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.36e-03, 9.11e-02, 2.64e-01, 1.00e-01, 3.73e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06616690784097522
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.69e-04, 6.61e-03, 1.42e-02, 2.45e-02, 5.62e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10953541568081857
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.97e-